In [1]:
%cd ..
import os, sys
sys.path.append(os.path.join(os.getcwd(), "src"))

d:\Blockchain\data\orai\orchai_validator


In [2]:
from orchai.tools import query, get_spark, psql_connect
from orchai.upload import sampling
from omegaconf import OmegaConf
import pyspark.sql.functions as F

## Fetch block height

In [3]:
spark = get_spark()

In [3]:
config = OmegaConf.load("config/etl_file_1m.yaml")

In [5]:
df = sampling(spark, config, from_block=7059473, to_block=9583823)

In [6]:
df.printSchema()

root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [7]:
Dict_Null = {
    c : df.filter(
        F.col(c).contains('None') | \
        F.col(c).contains('NULL') | \
        (F.col(c) == '')  | \
        F.col(c).isNull()
    ).count() 
    for c in df.columns
}

In [8]:
Dict_Null

{'block_height': 0,
 'operator_address': 0,
 'jailed': 0,
 'status': 0,
 'tokens': 0,
 'commission_rate': 0,
 'delegator_shares': 0,
 'self_bonded': 0,
 'propose': 0,
 'vote': 0}

In [4]:
conn = psql_connect(**config.src)

In [5]:
cur = conn.cursor()

In [33]:
import pandas as pd


def clone(config, start, end):
    with open("config/validators.txt", 'r') as f:
        validators = f.read().split()
        
    QUERY = f"""
        select * from {config.src.table} 
        where block_height >= {start} and block_height <= {end}
            and operator_address in ({", ".join([f"'{v}'" for v in validators])})
        order by block_height;
    """
    cur.execute(QUERY)
    return  pd.DataFrame(cur.fetchall(), 
                         columns=[
                            "block_height", "operator_address", "jailed", 
                            "status", "tokens", "commission_rate", "delegator_shares", 
                            "self_bonded", "propose", "vote"])

In [34]:
df = clone(config, 7059473, 9583823)

In [36]:
df.head()

,block_height,operator_address,jailed,status,tokens,commission_rate,delegator_shares,self_bonded,propose,vote
0,7059473,oraivaloper1qv5jn7tueeqw7xqdn5rem7s09n7zletree...,False,BOND_STATUS_BONDED,6465693643,3.000000e+16,6465693643000000000000000000,200000000000000000000000000,False,True
1,7059473,oraivaloper1zwxeq68hunufljymhy0f0x7r2vvh9mu2hh...,False,BOND_STATUS_BONDED,256616624,3.000000e+16,256616624000000000000000000,119795626000000000000000000,False,True
2,7059473,oraivaloper1rqq57xt5r5pnuguffcrltnvkul7n0jdxxd...,False,BOND_STATUS_BONDED,10193633794,1.000000e+17,10193633794000000000000000000,2002000000000000000000000,False,True
3,7059473,oraivaloper1yc9nysml8dxy447hp3aytr0nssr9pd9a47...,False,BOND_STATUS_BONDED,11577917753,5.000000e+16,11579038006221220000000000000,188012051887984650000000000,False,True
4,7059473,oraivaloper1yujklf8xlcngsdylgqj352pj9x3yvrrwf3...,False,BOND_STATUS_BONDED,1006270000,3.000000e+16,1006270000000000000000000000,2000000000000000000000000,False,True


In [38]:
df.shape

(656370, 10)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656370 entries, 0 to 656369
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   block_height      656370 non-null  int64  
 1   operator_address  656370 non-null  object 
 2   jailed            656370 non-null  bool   
 3   status            656370 non-null  object 
 4   tokens            656370 non-null  object 
 5   commission_rate   656370 non-null  float64
 6   delegator_shares  656370 non-null  object 
 7   self_bonded       656370 non-null  object 
 8   propose           656370 non-null  bool   
 9   vote              656370 non-null  bool   
dtypes: bool(3), float64(1), int64(1), object(5)
memory usage: 36.9+ MB


In [27]:
df.columns

Index(['block_height', 'operator_address', 'jailed', 'status', 'tokens',
       'commission_rate', 'delegator_shares', 'self_bonded', 'propose',
       'vote'],
      dtype='object')

In [41]:
pd.set_option("display.max_colWidth", None)

In [42]:
df[(df["block_height"] >= 9041123) & (df["block_height"] <= 9058523)].head(50)

,block_height,operator_address,jailed,status,tokens,commission_rate,delegator_shares,self_bonded,propose,vote
515229,9041123,oraivaloper1djm07np8dzyg4et3d7dqtr3692l80nggvl0edh,True,BOND_STATUS_UNBONDED,16326895,3.000000e+16,16326895000000000000000000,0,False,False
515230,9041123,oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7s2xq,True,BOND_STATUS_UNBONDED,12304300,3.000000e+16,12304300000000000000000000,0,False,False
515231,9041123,oraivaloper109vcny07r3waj9sld4ejasjyal0rudskeax7uc,True,BOND_STATUS_UNBONDED,2293369056,5.300000e+17,2293369056000000000000000000,0,False,False
515232,9041123,oraivaloper1qv5jn7tueeqw7xqdn5rem7s09n7zletreera88,False,BOND_STATUS_BONDED,22090778953,3.000000e+16,22090778953000000000000000000,768440000000000000000000000,False,True
515233,9041123,oraivaloper1zwxeq68hunufljymhy0f0x7r2vvh9mu2hh8nt8,False,BOND_STATUS_BONDED,4532605862,3.000000e+16,4532605862000000000000000000,119795626000000000000000000,False,True
515234,9041123,oraivaloper1rqq57xt5r5pnuguffcrltnvkul7n0jdxxdgey0,False,BOND_STATUS_BONDED,10211563063,1.000000e+17,10211563063000000000000000000,2002000000000000000000000,False,True
515235,9041123,oraivaloper1yc9nysml8dxy447hp3aytr0nssr9pd9a47l7gx,False,BOND_STATUS_BONDED,8266863381,3.000000e+16,8267663256823944000000000000,371139275127007200000000000,False,True
515236,9041123,oraivaloper1yujklf8xlcngsdylgqj352pj9x3yvrrwf3spc8,False,BOND_STATUS_BONDED,4107189280,3.000000e+16,4107599865491482000000000000,4000199939412869000000000,False,True
515237,9041123,oraivaloper1xesqr8vjvy34jhu027zd70ypl0nnev5euy9nyl,False,BOND_STATUS_BONDED,147136676200,3.000000e+16,147136676200000000000000000000,6378650000000000000000000000,False,True
515238,9041123,oraivaloper1f6q9wjn8qp3ll8y8ztd8290vtec2yxyxxygyy2,False,BOND_STATUS_BONDED,85476030779,3.000000e+16,85476030779000000000000000000,489930000000000000000000000,False,True
